In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Aug 03 10:00:28 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,656842,35.1,1435137,76.7,1435137,76.7
Vcells,1224853,9.4,8388608,64.0,1924961,14.7


In [3]:
PARAM <- list()
PARAM$experimento <- 50012

In [4]:
# Paso 1: Instalación de paquetes necesarios
if (!require("lightgbm")) install.packages("lightgbm")
if (!require("mlrMBO")) install.packages("mlrMBO")
if (!require("data.table")) install.packages("data.table")
if (!require("DiceKriging")) install.packages("DiceKriging")
if (!require("pROC")) install.packages("pROC")
library(DiceKriging)
library(lightgbm)
library(mlrMBO)
library(data.table)
library(parallel)
library(smoof)
library(mlr)
library(pROC)

Loading required package: lightgbm

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate

Loading required package: data.table

Loading required package: DiceKriging


Attaching package: ‘DiceKriging’


The following object is masked from ‘package:checkmate’:

    checkNames


Loading required package: pROC

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var




In [5]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("Cuantico", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [6]:
df <- fread("/content/datasets/creditcard.csv")

df[, Class := as.integer(Class)]
cat("Distribución original (positivo = fraude = 1):\n")
print(df[, .N, by = Class])

Distribución original (positivo = fraude = 1):
   Class      N
   <int>  <int>
1:     0 284315
2:     1    492


In [7]:
# Estratificación y separación de train/test
set.seed(1001111)
df[, idx := .I]  # agregamos índice para rearmar luego
train_idx <- df[, .SD[sample(.N * 0.7)], by = Class]$idx
train_full <- df[J(train_idx), on = "idx"]
test_full  <- df[!J(train_idx), on = "idx"]

test <- test_full

# Columnas predictoras
features <- setdiff(names(train_full), c("Class", "idx"))

In [8]:
optimizar_y_entrenar <- function(dataset, nombre_escenario, test_set) {
    
  dataset <- copy(dataset)
  dataset[, idx := NULL]
  
  dtrain <- lgb.Dataset(data = data.matrix(dataset[, ..features]), label = dataset$Class)
  
  param_base <- list(objective = "binary", metric = "auc", verbosity = -1, feature_pre_filter = FALSE)
  
  ps <- makeParamSet(
    makeNumericParam("learning_rate", lower = 0.01, upper = 0.2),
    makeIntegerParam("num_leaves", lower = 10L, upper = 200L),
    makeIntegerParam("min_data_in_leaf", lower = 5L, upper = 20L)
  )
  
  obj_fun <- makeSingleObjectiveFunction(
    fn = function(x) {
      x <- as.list(x)
      param <- modifyList(param_base, x)
      cv <- lgb.cv(params = param, data = dtrain, nfold = 3, nrounds = 100, early_stopping_rounds = 10, verbose = -1)
      return(max(unlist(cv$record_evals$valid$auc$eval)))
    },
    par.set = ps, minimize = FALSE
  )
  
  ctrl <- makeMBOControl()
  ctrl <- setMBOControlTermination(ctrl, iters = 30)
  ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())
  
  surrogate <- makeLearner("regr.km", predict.type = "se")
  run <- suppressMessages(suppressWarnings(mbo(obj_fun, learner = surrogate, control = ctrl)))
  
  # Entrenar modelo final
  best_param <- modifyList(param_base, as.list(run$x))
  modelo <- lgb.train(params = best_param, data = dtrain, nrounds = 100, verbose = -1)
  
  # Predecir en test set
  yhat <- predict(modelo, data.matrix(test_set[, ..features]))
  roc_obj <- pROC::roc(test_set$Class, yhat)
  auc_test <- pROC::auc(roc_obj)
  
  return(data.table(
    escenario = nombre_escenario,
    learning_rate = run$x$learning_rate,
    num_leaves = run$x$num_leaves,
    min_data_in_leaf = run$x$min_data_in_leaf,
    auc_test = as.numeric(auc_test)
  ))
}

In [9]:
# Escenario original
res_original <- optimizar_y_entrenar(train_full, "original", test_full)


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 
  - parameters upper bounds :  0.3591535 352 30 
  - best initial criterion value(s) :  37.0684 

N = 3, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=      -37.068  |proj g|=   0.00010818
At iterate     1  f =      -37.068  |proj g|=    0.00010943

iterations 1
function evaluations 2
segments explored during Cauchy searches 1
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 0.000109433
final function value -37.0684

F = -37.0684
final  value -37.068402 
converged

optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covarian

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [10]:
# Escenario con duplicación de positivos como negativos
duplicados <- copy(train_full[Class == 1])
duplicados[, Class := 0]
train_duplicado <- rbind(train_full, duplicados)

In [11]:
res_duplicado <- optimizar_y_entrenar(train_duplicado, "cuantico", test_full)


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 
  - parameters upper bounds :  0.3547314 354 28 
  - best initial criterion value(s) :  28.53223 

N = 3, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=      -28.532  |proj g|=      0.13394
At iterate     1  f =      -28.689  |proj g|=       0.14058
At iterate     2  f =      -28.689  |proj g|=      0.036837
At iterate     3  f =      -28.689  |proj g|=      0.030583
At iterate     4  f =      -28.689  |proj g|=       0.03335
At iterate     5  f =      -28.689  |proj g|=      0.092433
At iterate     6  f =      -28.689  |proj g|=      0.093178
At iterate     7  f =       -28.69  |proj g|=      0.094438
At iterate     8  f =      -28.691  |proj g|=      0.096519
At iterate     9

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [12]:
# Combinar
resumen <- rbind(res_original, res_duplicado)
print(resumen)

   escenario learning_rate num_leaves min_data_in_leaf  auc_test
      <char>         <num>      <int>            <int>     <num>
1:  original    0.13348701        135                6 0.4995721
2:  cuantico    0.01000581         35               20 0.9471995
